<a href="https://colab.research.google.com/github/rodeo9000/Traffic-Sign-AI/blob/main/TSRNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount ('/content/drive')

import os
import cv2 as cv
import numpy as np
import pandas as pd 
from PIL import Image
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.utils import to_categorical
from sklearn.metrics import accuracy_score
from keras.models import Sequential, load_model
from sklearn.model_selection import train_test_split
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout

Mounted at /content/drive


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sun Jan 29 21:57:58 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    50W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
#Get data
path = os.getcwd()
print (path)

data = '/content/drive/My Drive/Colab Notebooks/traffic sign classification/Data'
if os.path.exists(data) == True:
  print ("Data found")
else:
  print ("Error: Data not found")

/content
Data found


In [ ]:
#Unzip Compressed Data
!unzip '/content/drive/My Drive/Colab Notebooks/traffic sign classification/Data/DataTrain.zip' > /dev/null
!unzip '/content/drive/My Drive/Colab Notebooks/traffic sign classification/Data/DataTest.zip' > /dev/null

In [ ]:
dataset = []
names = []
types = 43

#Retrieving and preprocess the images
for i in range(types):
  path = os.getcwd()
  path = os.path.join(path,'Train',str(i))
  images = os.listdir(path)
  imagenum = 0

  for j in images:
    image = Image.open(path + '/'+ j)
    image = image.resize((40,40))
    image = np.array(image)
    dataset.append(image)
    names.append(i)
    imagenum += 1

#Save Data and Labels
np.save('/content/drive/My Drive/Colab Notebooks/traffic sign classification/Processed/dataset',dataset)
np.save('/content/drive/My Drive/Colab Notebooks/traffic sign classification/Processed/names',names)

In [ ]:
#Load Data
dataset = np.load('/content/drive/My Drive/Colab Notebooks/traffic sign classification/Processed/dataset.npy')
names = np.load('/content/drive/My Drive/Colab Notebooks/traffic sign classification/Processed/names.npy')

if dataset.shape[0] == names.shape[0]:
    print ('Successful')
else:
    print ('Error with data processing')

Successful


In [ ]:
#85% training, 15% testing
Xtr, Xts, Ytr, Yts = train_test_split(dataset, names, test_size = 0.15, random_state = 64)

#One-hot encode into vectors
Ytr = to_categorical(Ytr, 43)
Yts = to_categorical(Yts, 43)

#Convolutional Neural Network
model = Sequential()
model.add(Conv2D(filters = 36, kernel_size = (5,5), activation = 'relu', input_shape = Xtr.shape[1:]))
model.add(Conv2D(filters = 36, kernel_size = (5,5), activation = 'relu'))
model.add(MaxPool2D(pool_size = (2, 2)))
model.add(Dropout(rate = 0.2))
model.add(Conv2D(filters = 72, kernel_size = (3, 3), activation = 'relu'))
model.add(Conv2D(filters = 72, kernel_size = (3, 3), activation = 'relu'))
model.add(MaxPool2D(pool_size = (2, 2)))
model.add(Dropout(rate = 0.2))
model.add(Flatten())
model.add(Dense(256, activation = 'relu'))
model.add(Dropout(rate = 0.45))
model.add(Dense(43, activation = 'softmax'))

In [ ]:
#Compile Model
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 36, 36, 36)        2736      
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 36)        32436     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 16, 16, 36)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 16, 16, 36)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 14, 14, 72)        23400     
                                                                 
 conv2d_3 (Conv2D)           (None, 12, 12, 72)        46728     
                                                        

In [ ]:
#Training the model
model.fit(Xtr, Ytr, batch_size = 128, epochs = 12, validation_data = (Xts, Yts))

Epoch 1/12
261/261 [==============================] - 12s 9ms/step - loss: 2.4840 - accuracy: 0.4952 - val_loss: 0.5705 - val_accuracy: 0.8631
Epoch 2/12
261/261 [==============================] - 2s 7ms/step - loss: 0.5756 - accuracy: 0.8358 - val_loss: 0.1811 - val_accuracy: 0.9577
Epoch 3/12
261/261 [==============================] - 2s 7ms/step - loss: 0.3260 - accuracy: 0.9059 - val_loss: 0.1189 - val_accuracy: 0.9723
Epoch 4/12
261/261 [==============================] - 2s 7ms/step - loss: 0.2348 - accuracy: 0.9315 - val_loss: 0.0807 - val_accuracy: 0.9779
Epoch 5/12
261/261 [==============================] - 2s 7ms/step - loss: 0.1817 - accuracy: 0.9460 - val_loss: 0.0802 - val_accuracy: 0.9781
Epoch 6/12
261/261 [==============================] - 2s 7ms/step - loss: 0.1806 - accuracy: 0.9481 - val_loss: 0.0649 - val_accuracy: 0.9835
Epoch 7/12
261/261 [==============================] - 2s 7ms/step - loss: 0.1382 - accuracy: 0.9599 - val_loss: 0.0509 - val_accuracy: 0.9874
Epoch

In [ ]:
#Save the model to drive
model.save('/content/drive/My Drive/Colab Notebooks/traffic sign classification/Trained Model/CNN.h5')

In [ ]:
#Load model
model = load_model('/content/drive/My Drive/Colab Notebooks/traffic sign classification/Trained Model/CNN.h5')

In [ ]:
#Testing
Yts = pd.read_csv('/content/drive/My Drive/Colab Notebooks/traffic sign classification/Data/Test.csv')
names = Yts["ClassId"].values
images = Yts["Path"].values
dataset = []

for image in images:
    image = Image.open(image)
    image = image.resize((40,40))
    dataset.append(np.array(image))

Xts = np.array(dataset)
prediction = model.predict(Xts) 
classes = np.argmax(prediction, axis=1)

395/395 [==============================] - 1s 2ms/step


In [ ]:
print (accuracy_score(names, classes))

0.9650039588281869
